<a href="https://colab.research.google.com/github/MekhzZ/Learning_path_Data_Analyst/blob/main/Data_Wrangling_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Wrangling Advanced**

In [1]:
#loading module

import numpy as np
import pandas as pd

# Grouping data

In [2]:
df = pd.read_csv('/content/500_Person_Gender_Height_Weight_Index.csv')

In [3]:
df.shape

(500, 4)

In [5]:
#groupby getting row counts

df.groupby('Gender').size()

Gender
Female    255
Male      245
dtype: int64

In [6]:
df.Gender.value_counts()

Gender
Female    255
Male      245
Name: count, dtype: int64

In [8]:
#goupby using multiple columns

df.groupby(['Gender','Index']).mean()

Height      Weight
Gender Index                        
Female 0      186.571429   51.857143
       1      184.142857   58.714286
       2      171.682927   66.926829
       3      176.944444   87.722222
       4      174.338028  108.070423
       5      161.569892  135.526882
Male   0      188.666667   51.500000
       1      185.066667   59.733333
       2      178.035714   72.250000
       3      174.906250   85.937500
       4      173.322034  107.813559
       5      160.457143  130.552381

# windowing function

These are functions that operate on a set of rows (windows) but unlike the aggregate fucntions in grouping operatiion, the rows do not become seggregate into one output rpw and instead retain their separate identities

Window functions, also known as analytic functions, are a feature in SQL that allow you to perform calculations across a set of table rows that are related to the current row. Unlike aggregate functions, which return a single result for a group of rows, window functions do not cause rows to be grouped into a single output row. Instead, they perform calculations across a set of table rows that are related to the current row, allowing you to retain the individual rows in the result set.

In [10]:
# cumulative sum using expanding windowing function

l1 = [['x',10],['y',20],['z',30],['a',40]]
name_age = pd.DataFrame(l1,columns=['name','age'])
name_age


,name,age
0,x,10
1,y,20
2,z,30
3,a,40


In [19]:
name_age['cumulative_sum'] = name_age['age'].expanding().sum()
name_age.drop('sum',axis=1,inplace=True)
name_age

,name,age,cumulative_sum
0,x,10,10.0
1,y,20,30.0
2,z,30,60.0
3,a,40,100.0


In [20]:
#rolling window

window_size = 2
name_age['rolling_sum'] = name_age['age'].rolling(window_size).sum()
name_age

,name,age,cumulative_sum,rolling_sum
0,x,10,10.0,NaN
1,y,20,30.0,30.0
2,z,30,60.0,50.0
3,a,40,100.0,70.0


In [21]:
#using windowing functions with group by

l1 = [['x',10],['x',15],['y',17],['y',20],['z',25],['z',30],['a',40]]
name_age = pd.DataFrame(l1,columns=['name','age'])
name_age

,name,age
0,x,10
1,x,15
2,y,17
3,y,20
4,z,25
5,z,30
6,a,40


In [23]:
name_age.groupby('name').rolling(window_size)['age'].sum()

name   
a     6     NaN
x     0     NaN
      1    25.0
y     2     NaN
      3    37.0
z     4     NaN
      5    55.0
Name: age, dtype: float64

# Apply functions

In [27]:
#using apply for column sum

test = df[['Height','Weight']].head()
test.apply(np.sum, axis = 0)

Height    892
Weight    458
dtype: int64

In [28]:
test.apply(np.sum, axis = 1) # for row sum

0    270
1    276
2    295
3    299
4    210
dtype: int64

In [29]:
test

,Height,Weight
0,174,96
1,189,87
2,185,110
3,195,104
4,149,61


In [32]:
# using apply with user-defined functions

def cust_func(x):
  return x/10

#test.apply(cust_func)
test.apply(lambda x: x/10, axis = 0)

,Height,Weight
0,17.4,9.6
1,18.9,8.7
2,18.5,11.0
3,19.5,10.4
4,14.9,6.1


# Reshaping data


In [33]:
#long data

long_data = df.groupby(['Gender','Index']).mean()
long_data

Height      Weight
Gender Index                        
Female 0      186.571429   51.857143
       1      184.142857   58.714286
       2      171.682927   66.926829
       3      176.944444   87.722222
       4      174.338028  108.070423
       5      161.569892  135.526882
Male   0      188.666667   51.500000
       1      185.066667   59.733333
       2      178.035714   72.250000
       3      174.906250   85.937500
       4      173.322034  107.813559
       5      160.457143  130.552381

In [34]:
#using unstack to convert long data to wide

wide_data = long_data.unstack(level=0)
wide_data

Height                  Weight            
Gender      Female        Male      Female        Male
Index                                                 
0       186.571429  188.666667   51.857143   51.500000
1       184.142857  185.066667   58.714286   59.733333
2       171.682927  178.035714   66.926829   72.250000
3       176.944444  174.906250   87.722222   85.937500
4       174.338028  173.322034  108.070423  107.813559
5       161.569892  160.457143  135.526882  130.552381

In [38]:
# using stack to convert wide to long

wide_data.stack(1)

Height      Weight
Index Gender                        
0     Female  186.571429   51.857143
      Male    188.666667   51.500000
1     Female  184.142857   58.714286
      Male    185.066667   59.733333
2     Female  171.682927   66.926829
      Male    178.035714   72.250000
3     Female  176.944444   87.722222
      Male    174.906250   85.937500
4     Female  174.338028  108.070423
      Male    173.322034  107.813559
5     Female  161.569892  135.526882
      Male    160.457143  130.552381

In [40]:
## Using pivot to convert from long to wide

long_data.reset_index().pivot(index = 'Gender', columns = 'Index', values = 'Height')

Index,0,1,2,3,4,5
Gender,,,,,,
Female,186.571429,184.142857,171.682927,176.944444,174.338028,161.569892
Male,188.666667,185.066667,178.035714,174.906250,173.322034,160.457143
